# Orders API - Sentinel Hub Hosting

This notebook demonstrates using the hosting block in the Planet Orders API to deliver data to Sentinel Hub collections.

- Creating PlanetScope orders and delivering data to Sentinel Hub
  - Creating the order payloads
  - Submitting the payload to create the order
  - Checking the status of order

In [54]:
import os

# Planet Python SDK
import planet
if float(planet.__version__) < 2.8: print("Please update Planet python package to 2.8 or greater")

import logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

## Creating PlanetScope orders and delivering data to Sentinel Hub

We will order a handful of SkySatCollect items. We will split the items into two orders to demonstrate how to re-use an existing collection.

In [61]:
# Set up authentication
pl_api_key = os.getenv("PL_API_KEY")
auth = planet.Auth.from_key(pl_api_key)

In [50]:
from planet import Auth, order_request, OrdersClient
import os

auth = Auth.from_key(os.getenv("PL_API_KEY"))

# Define the items and products to order
# This example uses SkySat items and the pansharpened_udm2 bundle
item_ids_order1 = ["20240526_205646_ssc9_u0002", "20240526_205646_ssc9_u0001", "20240526_205624_ssc9_u0002", "20240526_205624_ssc9_u0001", "20240526_155658_ssc2_u0002"]
item_ids_order2 = ["20240526_155658_ssc2_u0001", "20240526_155647_ssc2_u0002", "20240526_155647_ssc2_u0001", "20240526_155639_ssc2_u0001", "20240526_155547_ssc2_u0001"]
products_order1 = [order_request.product(item_ids_order1, 'pansharpened_udm2', 'SkySatCollect')]
products_order2 = [order_request.product(item_ids_order2, 'pansharpened_udm2', 'SkySatCollect')]

# build the first request, note that hosting is set to "sentinel_hub" and no collection_id id provided, meaning
# a new collection will be created on your behalf
request_1 = order_request.build_request("order-skysat-to-sentinel-hub-1", products=products_order1, hosting="sentinel_hub")

# orders after they are created will be added to this list to use in other cells
orders = []

async with planet.Session(auth=auth) as sess:
    cl = OrdersClient(sess)

    # create the first order
    order1 = await cl.create_order(request_1)
    orders.append(order1)
    # access the collection_id created on your behalf
    collection_id = order1["hosting"]["sentinel_hub"]["collection_id"]
    print(f"Collection ID: {collection_id}")

    # use the collection_id from the first order to create the second order
    # if the items you are ordering are compatible will an existing collection, you can send more data to an existing collection
    request_2 = order_request.build_request(
        "order-skysat-to-sentinel-hub-2", products=products_order2, hosting="sentinel_hub", collection_id=collection_id
    )

    # create the second order
    order2 = await cl.create_order(request_2)
    orders.append(order2)

# print the orders for inspection (if desired)
for ord in orders:
    print(ord)


2024-06-10 13:13:16,237 - INFO - Session read timeout set to 30.0.
2024-06-10 13:13:16,252 - INFO - POST https://api.planet.com/compute/ops/orders/v2 - Sent
2024-06-10 13:13:21,444 - INFO - HTTP Request: POST https://api.planet.com/compute/ops/orders/v2 "HTTP/1.1 202 Accepted"
2024-06-10 13:13:21,444 - INFO - POST https://api.planet.com/compute/ops/orders/v2 - Status 202
2024-06-10 13:13:21,448 - INFO - POST https://api.planet.com/compute/ops/orders/v2 - Sent


Collection ID: 396da26c-56ec-415b-8343-9fcbec78c802


2024-06-10 13:13:23,916 - INFO - HTTP Request: POST https://api.planet.com/compute/ops/orders/v2 "HTTP/1.1 202 Accepted"
2024-06-10 13:13:23,917 - INFO - POST https://api.planet.com/compute/ops/orders/v2 - Status 202


{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/a7d3244a-b5ab-4d87-a92b-40ab3373ed8d'}, 'created_on': '2024-06-10T18:13:21.103Z', 'error_hints': [], 'hosting': {'sentinel_hub': {'collection_id': '396da26c-56ec-415b-8343-9fcbec78c802'}}, 'id': 'a7d3244a-b5ab-4d87-a92b-40ab3373ed8d', 'last_message': 'Preparing order', 'last_modified': '2024-06-10T18:13:21.103Z', 'name': 'order-skysat-to-sentinel-hub-1', 'products': [{'item_ids': ['20240526_205646_ssc9_u0002', '20240526_205646_ssc9_u0001', '20240526_205624_ssc9_u0002', '20240526_205624_ssc9_u0001', '20240526_155658_ssc2_u0002'], 'item_type': 'SkySatCollect', 'product_bundle': 'pansharpened_udm2'}], 'state': 'queued', 'tools': [{'file_format': {'format': 'COG'}}]}
{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/8da27807-2914-4713-8558-dd1b16f2a7de'}, 'created_on': '2024-06-10T18:13:23.599Z', 'error_hints': [], 'hosting': {'sentinel_hub': {'collection_id': '396da26c-56ec-415b-8343-9fcbec78c802'}}, 'id':

In [67]:
# Check the status of the orders - rerun this cell as much as you would like to check on the status
# of your orders. This will print out the order ID and the current state of each order.

async with planet.Session(auth=auth) as sess:
    cl = OrdersClient(sess)

    states = []
    for order in orders:
        order_details = await cl.get_order(ord["id"])
        order_state = order_details["state"]
        states.append(f"Order ID: {order['id']} ({order_state})")

# print out the results
print("------")
print(f"Collection ID: {collection_id}")
for state in states:
    print(state)

2024-06-10 13:51:04,029 - INFO - Session read timeout set to 30.0.
2024-06-10 13:51:04,042 - INFO - GET https://api.planet.com/compute/ops/orders/v2/8da27807-2914-4713-8558-dd1b16f2a7de - Sent
2024-06-10 13:51:04,132 - INFO - HTTP Request: GET https://api.planet.com/compute/ops/orders/v2/8da27807-2914-4713-8558-dd1b16f2a7de "HTTP/1.1 200 OK"
2024-06-10 13:51:04,133 - INFO - GET https://api.planet.com/compute/ops/orders/v2/8da27807-2914-4713-8558-dd1b16f2a7de - Status 200
2024-06-10 13:51:04,146 - INFO - GET https://api.planet.com/compute/ops/orders/v2/8da27807-2914-4713-8558-dd1b16f2a7de - Sent
2024-06-10 13:51:04,195 - INFO - HTTP Request: GET https://api.planet.com/compute/ops/orders/v2/8da27807-2914-4713-8558-dd1b16f2a7de "HTTP/1.1 200 OK"
2024-06-10 13:51:04,196 - INFO - GET https://api.planet.com/compute/ops/orders/v2/8da27807-2914-4713-8558-dd1b16f2a7de - Status 200


------
Collection ID: 396da26c-56ec-415b-8343-9fcbec78c802
Order ID: a7d3244a-b5ab-4d87-a92b-40ab3373ed8d (success)
Order ID: 8da27807-2914-4713-8558-dd1b16f2a7de (success)


When the orders are in a 'success' state, you can access the data you ordered through the Orders API in Sentinel Hub. You can view the data by going to the 'Tiles' tab for the created collection. The collection name will be the name of the first order (collections inherit the order name if not created manually in the UI).  
Url: https://apps.sentinel-hub.com/dashboard/#/collections/collection/{your_collection_id}/